In [2]:
import pandas as pd
import glob
import os
import json
import sys
from collections import Counter
import numpy as np
pd.set_option('max_colwidth',200)
pd.set_option('max_rows',500)

In [10]:
imgs = pd.read_csv('all_responses_round0-3_cleaned.csv',sep="\t")

In [11]:
imgs.columns

Index(['Unnamed: 0', 'vg_img_id', 'cat', 'synset', 'vg_obj_name',
       'responses_r0', 'opt-outs', 'vg_domain', 'top_response_domain_r0',
       'responses_domains_r0', 'url', 'sample_type', 'responses_domains_r1',
       'responses_r1', 'top_response_domain_r1', 'responses_domains_r2',
       'responses_r2', 'top_response_domain_r2', 'responses_domains_r3',
       'responses_r3', 'top_response_domain_r3', 'vg_object_id',
       'all_responses', 'clean', 'canon', 'spellchecked'],
      dtype='object')

In [15]:
imgs[['spellchecked','url']].head(50)

,spellchecked,url
0,"{'man': 11, 'batter': 8, 'baseball player': 2, 'player': 5, 'person': 4, 'helmet': 2, 'boy': 1, 'uniform': 1, 'bat': 1}",http://object-naming-amore.upf.edu//2357077_815223_singleton_obj.png
1,"{'jacket': 14, 'sweatshirt': 4, 'man': 11, 'hoodie': 4, 'person': 2, 'shirt': 1}",http://object-naming-amore.upf.edu//2378759_1365136_supercat_unique.png
2,"{'giraffe': 32, 'woman': 1}",http://object-naming-amore.upf.edu//2319953_3089834_singleton_obj.png
3,{'giraffe': 36},http://object-naming-amore.upf.edu//2361401_2363103_singleton_obj.png
4,"{'sandwich': 26, 'food': 7, 'hotdog': 1, 'cake': 1, 'burger': 1}",http://object-naming-amore.upf.edu//2401994_1141828_singleton_obj.png
5,"{'van': 15, 'automobile': 3, 'car': 10, 'tailgate': 1, 'truck': 2, 'bus': 1, 'vehicle': 2, 'suitcase': 1, 'motorized vehicle': 1}",http://object-naming-amore.upf.edu//2415944_3255495_singleton_obj.png
6,"{'car': 33, 'automobile': 1, 'vehicle': 1, 'van': 1}",http://object-naming-amore.upf.edu//2398394_1180292_seed_ambiguous.png
7,"{'desk': 29, 'curtain': 1, 'keyboard': 3, 'computer desk': 1, 'electronics': 1}",http://object-naming-amore.upf.edu//1159907_1598944_singleton_obj.png
8,"{'zebra': 33, 'horse': 1}",http://object-naming-amore.upf.edu//2362791_771078_singleton_obj.png
9,"{'child': 14, 'boy': 11, 'skater': 1, 'person': 2, 'kid': 3, 'skateboarder': 2, 'glass': 2}",http://object-naming-amore.upf.edu//2411103_360540_singleton_obj.png


In [ ]:
#pd.crosstab(df.top_response_domain_r1,df.top_response_domain_r2)

In [20]:
pairs = pd.read_csv('names_pairs_relations_v2.csv',index_col=0)
print(len(pairs))
print(pairs.columns)
print(pairs.relation.value_counts()/len(pairs))
print(pairs.relation.value_counts())

123502
Index(['word1', 'word2', 'relation', 'totalfreq', 'freq-w1-w2', 'freq-w2-w1'], dtype='object')
crossclassified    0.640969
not-covered        0.298821
hypernymy.1        0.019085
co-hyponymy        0.018753
hypernymy.2        0.016858
synonymy           0.005514
Name: relation, dtype: float64
crossclassified    79161
not-covered        36905
hypernymy.1         2357
co-hyponymy         2316
hypernymy.2         2082
synonymy             681
Name: relation, dtype: int64


In [21]:
#pairs[pairs.totalfreq].sort_values(['relation','freq_both_orders'],ascending=[True, False])
d=pairs[(pairs.totalfreq>100) & (pairs.relation=="not-covered")] #.sort_values('freq_w1_w2',ascending=False)
#d.to_csv("crossclassified.csv")
print(len(d))
d.head(100)

32


,word1,word2,relation,totalfreq,freq-w1-w2,freq-w2-w1
983,train,train car,not-covered,323,323,0
2234,jacket,suit jacket,not-covered,242,240,2
1352,counter,kitchen counter,not-covered,241,241,0
2236,suit,suit jacket,not-covered,237,236,1
3905,food truck,truck,not-covered,191,0,191
1357,countertop,kitchen counter,not-covered,190,166,24
38,hoodie,jacket,not-covered,184,27,157
1361,kitchen counter,table,not-covered,172,39,133
834,pizza,pizza slice,not-covered,169,169,0
5622,train,train engine,not-covered,168,168,0


In [43]:
df = pd.read_csv('domains_names_pairs_relations_v2.csv',index_col=0)
#b1=df.str.startswith('man')
df.columns=['domain', 'word1', 'word2', 'relation', 'totalfreq', 'freq_w1_w2',
       'freq_w2_w1', 'domain_w1', 'domain_w2', 'syn_w1', 'syn_w2']
df.head()
#df[df.word1=="man"].sort_values('totalfreq',ascending=False)
#df[(df.word1=="man") & (df.word2=="shirt")]
#df[df.freq_w2_w1 > df.freq_w1_w2]
df[df.relation=="crossclassified"].syn_w1.value_counts()
df[df.relation=="crossclassified"].syn_w2.value_counts()


0    59496
Name: syn_w2, dtype: int64